# Analyse intégrée du RSV (France, 2018-2027)

Ce carnet rassemble toutes les étapes de l'analyse menée dans le cadre de la thèse, en suivant une trame méthodologique claire :

1. **Préparation des données** : harmonisation hebdomadaire des sources sanitaires et comportementales.
2. **Modélisation explicative (OLS)** : identification des déterminants et quantification de leurs effets moyens.
3. **Scénarios contrefactuels** : projection de trajectoires alternatives (vaccination, gestes barrières).
4. **Série interrompue (ITS)** : caractérisation des ruptures liées à la pandémie.
5. **SARIMAX multivarié** : prolongement de la série observée jusqu'en 2027.
6. **Prévisions univariées** (5 saisons) et scénarios dérivés.
7. **Diagnostics & exports** pour réutilisation (Streamlit, annexes).

Chaque section rappelle la logique, détaille les étapes techniques et met en avant les résultats à retenir.

In [49]:
from pathlib import Path
import warnings
from itertools import product

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.statespace.sarimax import SARIMAX

from IPython.display import display

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 180)
np.random.seed(42)

px.defaults.template = "plotly_white"
px.defaults.width = 1000
px.defaults.height = 520

DATA = Path("../data_clean")
FILES = {
    "common_FR_long": DATA / "ODISSEE/common_FR_long.csv",
    "vacsi_fr_extended": DATA / "VACSI/vacsi_fr_extended.csv",
    "google_mobility_fr_weekly": DATA / "GOOGLE/google_mobility_fr_weekly.csv",
    "coviprev_reg_weekly": DATA / "COVIPREV/coviprev_reg_weekly.csv",
    "meteo_fr_weekly": DATA / "METEO/meteo_fr_weekly.csv",
    "erviss_fr_weekly": DATA / "ERVISS/erviss_fr_weekly.csv",
}

missing = [k for k, p in FILES.items() if not p.exists()]
if missing:
    raise FileNotFoundError(f"Fichiers manquants: {missing}")
else:
    print("✅ Tous les fichiers nécessaires sont disponibles.")

COVID_START = pd.Timestamp("2020-03-01")
VACC_START = pd.Timestamp("2021-01-01")

LAG_VACC, LAG_MNP, LAG_WORK = 4, 8, 9
SEASON_PERIOD = 52
print(f"⏱️ COVID_START={COVID_START.date()}, VACC_START={VACC_START.date()} | Lags initiaux: {LAG_VACC, LAG_MNP, LAG_WORK}")


✅ Tous les fichiers nécessaires sont disponibles.
⏱️ COVID_START=2020-03-01, VACC_START=2021-01-01 | Lags initiaux: (4, 8, 9)


In [50]:
def keyify(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    iso = pd.to_datetime(df["date_monday"]).dt.isocalendar()
    df["year_iso"] = iso["year"].astype(int)
    df["week_iso_num"] = iso["week"].astype(int)
    return df

def zscore(series: pd.Series) -> pd.Series:
    std = series.std(ddof=0)
    return (series - series.mean()) / std if std != 0 else series * 0

def build_time_features(df: pd.DataFrame, period: int = 52) -> pd.DataFrame:
    df = df.copy()
    df["t"] = np.arange(len(df))
    df["sin52"] = np.sin(2 * np.pi * df["t"] / period)
    df["cos52"] = np.cos(2 * np.pi * df["t"] / period)
    return df

def load_datasets(files: dict[str, Path]) -> dict[str, pd.DataFrame]:
    data = {}
    for name, path in files.items():
        data[name] = pd.read_csv(path)
        print(f"✅ {name} chargé ({data[name].shape[0]} lignes)")
    return data

def merge_exog(rsv_df: pd.DataFrame, vac_df: pd.DataFrame, work_df: pd.DataFrame, cov_df: pd.DataFrame) -> pd.DataFrame:
    merged = (
        rsv_df[["date_monday", "year_iso", "week_iso_num"]]
        .merge(vac_df, on=["year_iso", "week_iso_num"], how="left")
        .merge(work_df, on=["year_iso", "week_iso_num"], how="left")
        .merge(cov_df, on=["year_iso", "week_iso_num"], how="left")
        .set_index("date_monday")
        .sort_index()
    )
    return merged

def build_model_matrix(df: pd.DataFrame, lags: tuple[int, int, int], mask_vars: list[str] | None = None) -> pd.DataFrame:
    lag_vac, lag_mnp, lag_work = lags
    df = df.copy()
    df["work_red"] = zscore(-df["work"])
    if mask_vars:
        for var in mask_vars:
            df[var] = zscore(df[var])
        df["MNP_score"] = df[mask_vars + ["work_red"]].mean(axis=1)
    else:
        df["MNP_score"] = zscore(df["work_red"])

    X = pd.DataFrame(index=df.index)
    X["cov12_lag"] = df["couv_complet"].shift(lag_vac)
    X["MNP_lag"] = df["MNP_score"].shift(lag_mnp)
    X["work_lag"] = df["work"].shift(lag_work)
    return build_time_features(X)


def plot_series(df: pd.DataFrame, y_col: str = "RSV", fitted: dict[str, pd.Series] | None = None, title: str = "RSV observé vs ajusté") -> None:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df[y_col], name="RSV observé", mode="lines", line=dict(color="black")))
    if fitted:
        for name, series in fitted.items():
            fig.add_trace(go.Scatter(x=series.index, y=series, name=name, mode="lines", line=dict(dash="dot")))
    fig.add_vline(x=COVID_START, line_dash="dash", line_color="red")
    fig.add_vline(x=VACC_START, line_dash="dash", line_color="green")
    fig.update_layout(title=title, xaxis_title="Semaine", yaxis_title=y_col)
    fig.show()


## 1. Préparer les jeux de données

Chargement des sources harmonisées (ODISSEE, VACSI, Google Mobility, CoviPrev, Météo, ERVISS) et contrôle de cohérence avant modélisation.

In [ ]:
# ==========================================
# 🧱 BLOC 1 — Setup, Helpers & Chargement Données
# ==========================================

from pathlib import Path
import warnings
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.outliers_influence import variance_inflation_factor
from itertools import product
import plotly.express as px
import plotly.graph_objects as go

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 180)
np.random.seed(42)

px.defaults.template = "plotly_white"
px.defaults.width = 1000
px.defaults.height = 520

# ==========================================
# 📁 Chemins & Fichiers
# ==========================================
DATA = Path("../data_clean")
FILES = {
    "common_FR_long": DATA / "ODISSEE/common_FR_long.csv",
    "vacsi_fr_extended": DATA / "VACSI/vacsi_fr_extended.csv",
    "google_mobility_fr_weekly": DATA / "GOOGLE/google_mobility_fr_weekly.csv",
    "coviprev_reg_weekly": DATA / "COVIPREV/coviprev_reg_weekly.csv",
    "meteo_fr_weekly": DATA / "METEO/meteo_fr_weekly.csv",
    "erviss_fr_weekly": DATA / "ERVISS/erviss_fr_weekly.csv",
}

missing = [k for k, p in FILES.items() if not p.exists()]
if missing:
    raise FileNotFoundError(f"❌ Fichiers manquants: {missing}")
print("✅ Tous les fichiers nécessaires sont disponibles.")

COVID_START = pd.Timestamp("2020-03-01")
VACC_START  = pd.Timestamp("2021-01-01")

LAG_VACC, LAG_MNP, LAG_WORK = 4, 8, 9
SEASON_PERIOD = 52
print(f"⏱️ COVID_START={COVID_START.date()}, VACC_START={VACC_START.date()} | Lags: {LAG_VACC, LAG_MNP, LAG_WORK}")

# ==========================================
# 🧩 Fonctions utilitaires
# ==========================================
def keyify(df: pd.DataFrame) -> pd.DataFrame:
    iso = pd.to_datetime(df["date_monday"]).dt.isocalendar()
    df["year_iso"] = iso["year"].astype(int)
    df["week_iso_num"] = iso["week"].astype(int)
    return df

def zscore(s): return (s - s.mean()) / s.std(ddof=0) if s.std(ddof=0) != 0 else s * 0
def build_time_features(df, period=52):
    df = df.copy()
    df["t"] = np.arange(len(df))
    df["sin52"] = np.sin(2 * np.pi * df["t"] / period)
    df["cos52"] = np.cos(2 * np.pi * df["t"] / period)
    return df

def load_datasets(files):
    data = {}
    for name, path in files.items():
        data[name] = pd.read_csv(path)
        print(f"✅ {name} chargé ({data[name].shape[0]} lignes)")
    return data

def merge_exog(rsv_df, vac_df, work_df, cov_df):
    merged = (
        rsv_df[["date_monday", "year_iso", "week_iso_num"]]
        .merge(vac_df, on=["year_iso", "week_iso_num"], how="left")
        .merge(work_df, on=["year_iso", "week_iso_num"], how="left")
        .merge(cov_df, on=["year_iso", "week_iso_num"], how="left")
        .set_index("date_monday")
        .sort_index()
    )
    return merged

def build_model_matrix(df, lags=(4, 8, 9), mask_vars=None):
    lag_vac, lag_mnp, lag_work = lags
    df["work_red"] = zscore(-df["work"])
    if mask_vars:
        for v in mask_vars: df[v] = zscore(df[v])
        df["MNP_score"] = df[mask_vars + ["work_red"]].mean(axis=1)
    else:
        df["MNP_score"] = zscore(df["work_red"])

    X = pd.DataFrame(index=df.index)
    X["cov12_lag"] = df["couv_complet"].shift(lag_vac)
    X["MNP_lag"]   = df["MNP_score"].shift(lag_mnp)
    X["work_lag"]  = df["work"].shift(lag_work)
    return build_time_features(X)

def plot_series(df, y_col="RSV", y_fit=None, title="RSV Observé vs Ajusté"):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df[y_col], name="RSV observé", mode="lines", line=dict(color="black")))
    if y_fit is not None:
        fig.add_trace(go.Scatter(x=df.index, y=y_fit, name="Ajusté", mode="lines", line=dict(color="blue", dash="dot")))
    fig.update_layout(title=title, xaxis_title="Semaine", yaxis_title=y_col)
    fig.show()

# ==========================================
# 📊 Chargement des données
# ==========================================
data = load_datasets(FILES)

common = keyify(data["common_FR_long"])
mask = (common["topic"] == "RSV") & (common["geo_level"] == "FR")
age_used = next(a for a in ["00-04 ans", "0-1 an", "Tous âges"] if ((mask) & (common["classe_d_age"] == a)).any())
mask &= (common["classe_d_age"] == age_used)

ycol = "taux_passages_urgences" if "taux_passages_urgences" in common.columns else "taux_sos"
rsv = common.loc[mask, ["date_monday", "year_iso", "week_iso_num", ycol]].rename(columns={ycol: "RSV"})
rsv["date_monday"] = pd.to_datetime(rsv["date_monday"])
rsv = rsv.sort_values("date_monday")
print(f"✅ RSV prêt ({age_used}) — {rsv.shape[0]} lignes")

vac = keyify(data["vacsi_fr_extended"]).query("geo_level=='FR'")[["year_iso","week_iso_num","couv_complet"]]
gm  = keyify(data["google_mobility_fr_weekly"])
work = gm.query("geo_level=='FR' & indicator=='workplaces'")[["year_iso","week_iso_num","value"]].rename(columns={"value": "work"})
cov = keyify(data["coviprev_reg_weekly"])
mask_vars = ["port_du_masque","lavage_des_mains","aeration_du_logement","saluer_sans_serrer_la_main"]
cov_nat = cov[cov["indicator"].isin(mask_vars)].groupby(["year_iso","week_iso_num","indicator"])["value"].mean().unstack()
print(f"✅ CoviPrev agrégé nationalement ({len(cov_nat)} semaines)")

X_base = merge_exog(rsv, vac, work, cov_nat)
X_full = build_model_matrix(X_base, lags=(LAG_VACC, LAG_MNP, LAG_WORK), mask_vars=mask_vars)

df_base = rsv.set_index("date_monday")[["RSV"]].join(X_full, how="left").dropna().sort_index()
print(f"✅ Base finale prête : {df_base.shape}")


✅ Tous les fichiers nécessaires sont disponibles.
⏱️ COVID_START=2020-03-01, VACC_START=2021-01-01 | Lags: (4, 8, 9)
✅ common_FR_long chargé (3223 lignes)
✅ vacsi_fr_extended chargé (105 lignes)
✅ google_mobility_fr_weekly chargé (840 lignes)
✅ coviprev_reg_weekly chargé (1296 lignes)
✅ meteo_fr_weekly chargé (392 lignes)
✅ erviss_fr_weekly chargé (11713 lignes)


### 4.1 Première vérification

On jette un coup d'œil :
- Aperçu des premières lignes pour vérifier les colonnes.
- Statistiques simples (moyenne, minimum, maximum) pour repérer un éventuel bug.
- Message clair si un fichier manque ou si une valeur paraît étrange.

Si tout est vert, on peut passer aux modèles.

In [ ]:
display(df_base.head())
summary = df_base.describe().T[["mean", "std", "min", "max"]].round(2)
display(summary)


,RSV,cov12_lag,MNP_lag,work_lag,t,sin52,cos52
date_monday,,,,,,,
2021-01-25,711.936545,0.000000,0.030364,-23.494505,56,0.464723,0.885456
2021-02-01,763.390974,0.000146,0.612450,-20.791209,57,0.568065,0.822984
2021-02-08,873.929009,0.001203,-0.068845,-20.241758,58,0.663123,0.748511
2021-02-15,982.824427,0.002820,1.880221,-19.362637,59,0.748511,0.663123
2021-02-22,1055.391309,0.022080,2.280871,-47.428571,60,0.822984,0.568065


,mean,std,min,max
RSV,822.19,628.81,170.03,2861.52
cov12_lag,12.55,6.75,0.00,17.49
MNP_lag,-0.27,0.73,-1.20,2.28
work_lag,-16.99,10.59,-53.20,-3.09
t,104.50,28.43,56.00,153.00
sin52,-0.00,0.73,-1.00,1.00
cos52,-0.06,0.69,-1.00,1.00


## 2. Modèle OLS : comprendre les déterminants

On démarre avec un modèle simple, puis on optimise les retards et on ajoute les interactions (vaccination × gestes) et la saisonnalité.

## 5. Modèle OLS : lire les relations simples

Le modèle OLS (régression linéaire) agit comme une loupe. Il répond à la question : "Quand la vaccination monte ou que les gestes barrières changent, comment réagit le RSV en moyenne ?"

On commence doucement puis on améliore le modèle étape par étape.

### 5.1 On part d'un modèle très simple

On garde seulement quelques ingrédients (vaccination décalée, gestes barrières résumés, saisonnalité) pour suivre la tendance globale.

In [ ]:
Y = df_base["RSV"].astype(float)
X_cols_base = ["cov12_lag", "MNP_lag", "work_lag", "sin52", "cos52"]
X_base_design = df_base[X_cols_base]

ols_base = sm.OLS(Y, sm.add_constant(X_base_design)).fit(cov_type="HC3")
print(f"R² ajusté = {ols_base.rsquared_adj:.3f} | AIC = {ols_base.aic:.1f} | Durbin-Watson = {sm.stats.stattools.durbin_watson(ols_base.resid):.3f}")

plot_series(
    df_base,
    fitted={"OLS base": ols_base.fittedvalues}
)


R² ajusté = 0.530 | AIC = 1473.0 | Durbin-Watson = 0.150


### 5.2 On teste plusieurs décalages

Les effets ne sont pas instantanés : une vaccination ou un changement de mobilité agit après quelques semaines. On lance donc une recherche automatique qui essaie plein de combinaisons de retards et garde celle qui colle le mieux aux données.

In [ ]:
best_r2, best_lags = -np.inf, (LAG_VACC, LAG_MNP, LAG_WORK)
lags_grid = [(lv, lm, lw) for lv in range(2, 9) for lm in range(4, 13) for lw in range(4, 13)]

for lv, lm, lw in lags_grid:
    X_tmp = build_model_matrix(X_base, lags=(lv, lm, lw), mask_vars=mask_vars)
    df_tmp = rsv.set_index("date_monday")["RSV"].to_frame().join(X_tmp).dropna()
    if len(df_tmp) < 40:
        continue
    fit_tmp = sm.OLS(df_tmp["RSV"], sm.add_constant(df_tmp[X_cols_base])).fit()
    if fit_tmp.rsquared_adj > best_r2:
        best_r2, best_lags = fit_tmp.rsquared_adj, (lv, lm, lw)

print(f"🥇 Retards optimaux identifiés : {best_lags} (R² ajusté ≈ {best_r2:.3f})")

X_full_opt = build_model_matrix(X_base, lags=best_lags, mask_vars=mask_vars)

df_opt = (
    rsv.set_index("date_monday")["RSV"].to_frame()
    .join(X_full_opt)
    .dropna()
)

meteo = keyify(data["meteo_fr_weekly"])[["year_iso", "week_iso_num", "tmean"]]
df_opt = (
    keyify(df_opt.reset_index())
    .merge(meteo, on=["year_iso", "week_iso_num"], how="left")
    .set_index("date_monday")
    .sort_index()
)

df_opt["tmean_z"] = zscore(df_opt["tmean"])
df_opt["vacc_x_mnp"] = df_opt["cov12_lag"] * df_opt["MNP_lag"]
df_opt["RSV_lag1"] = df_opt["RSV"].shift(1)
df_opt["RSV_lag2"] = df_opt["RSV"].shift(2)
df_opt = df_opt.dropna()

X_opt_cols = [
    "cov12_lag", "MNP_lag", "work_lag", "tmean_z",
    "vacc_x_mnp", "RSV_lag1", "RSV_lag2", "sin52", "cos52"
]
X_opt = df_opt[X_opt_cols]

ols_opt = sm.OLS(df_opt["RSV"], sm.add_constant(X_opt)).fit(cov_type="HC3")
print(f"OLS optimisé — R² ajusté = {ols_opt.rsquared_adj:.3f} | AIC = {ols_opt.aic:.1f} | Durbin-Watson = {sm.stats.stattools.durbin_watson(ols_opt.resid):.3f}")

LAG_MNP_EFFECT = 3
df_opt["MNP_lag_causal"] = df_opt["MNP_lag"].shift(LAG_MNP_EFFECT)
df_opt["vacc_x_mnp_causal"] = df_opt["cov12_lag"] * df_opt["MNP_lag_causal"]

X_causal_cols = [
    "cov12_lag", "MNP_lag_causal", "work_lag", "tmean_z",
    "vacc_x_mnp_causal", "RSV_lag1", "RSV_lag2", "sin52", "cos52"
]
Xo_causal = df_opt[X_causal_cols].dropna()

ols_causal = sm.OLS(
    df_opt.loc[Xo_causal.index, "RSV"],
    sm.add_constant(Xo_causal)
).fit(cov_type="HC3")
print(f"OLS causal (décalage MNP +{LAG_MNP_EFFECT} sem) — R² ajusté = {ols_causal.rsquared_adj:.3f} | AIC = {ols_causal.aic:.1f}")

fitted_curves = {
    "OLS optimisé": ols_opt.fittedvalues,
    "OLS causal": ols_causal.fittedvalues.reindex(df_opt.index)
}
plot_series(df_opt, fitted=fitted_curves, title="RSV — OLS optimisé vs causal")


🥇 Retards optimaux identifiés : (7, 12, 4) (R² ajusté ≈ 0.657)
OLS optimisé — R² ajusté = 0.968 | AIC = 1069.4 | Durbin-Watson = 1.960
OLS causal (décalage MNP +3 sem) — R² ajusté = 0.970 | AIC = 1029.3


### Diagnostics & interprétations

### Ce qu'il faut retenir tout de suite

- **OLS optimisé** explique presque toutes les variations et laisse des résidus propres.
- **OLS causal** ajoute un décalage sur les gestes barrières pour coller au comportement humain.
- **ITS** confirme les ruptures dues au COVID et au vaccin mais reste moins précis entre deux événements.
- **SARIMAX** est le meilleur compagnon pour les prévisions multi-années.

## 9. Zoom sur le modèle OLS optimisé

On traduit les diagnostics en phrases faciles :
- quels indicateurs globaux confirment que le modèle est solide,
- quelles variables pèsent le plus,
- comment vérifier que le modèle reste stable dans le temps.

L'idée est d'avoir un mode d'emploi, pas seulement un tableau de chiffres.

### Indicateurs globaux

- **R² ajusté ≈ 0,97** : le modèle explique presque tout ce qui se passe chaque semaine.
- **AIC/BIC ≈ 1 030** : on reste sur un modèle compact sans surenchère.
- **Durbin-Watson ≈ 1,98** : les résidus ressemblent à du bruit aléatoire (pas de structure cachée).
- Les tests complémentaires restent dans des zones acceptables : rien d'alarmant.

In [ ]:
vif_df = pd.DataFrame({
    "Variable": X_opt_cols,
    "VIF": [variance_inflation_factor(X_opt.values, i) for i in range(X_opt.shape[1])]
}).sort_values("VIF", ascending=False)

display(vif_df)

,Variable,VIF
5,RSV_lag1,66.713191
6,RSV_lag2,64.478941
3,tmean_z,8.277630
8,cos52,6.590143
1,MNP_lag,6.289709
4,vacc_x_mnp,5.760349
0,cov12_lag,3.952017
7,sin52,2.702864
2,work_lag,2.576776


👉 Ce que disent les coefficients :
- La vaccination fait baisser le RSV, surtout quand elle est combinée à des gestes barrières stables.
- Les gestes barrières retardés de 3 semaines capturent bien le temps nécessaire pour que les habitudes changent.
- La saisonnalité (sinus/cosinus) et l'inertie du RSV gardent la forme globale de la courbe.
- Le climat et la mobilité au travail jouent un rôle plus doux mais non négligeable.

## 3. Scénarios contrefactuels (modèle OLS causal)

Les scénarios modifient les niveaux de vaccination et de gestes barrières après le COVID. Pour “Sans MNP”, on impose le niveau de relâchement maximal observé ; pour “MNP +50 %”, on renforce durablement la protection. Bootstrap = incertitude sur l'écart cumulé.

## 10. Scénarios "et si..."

On joue avec les leviers principaux (vaccination et gestes barrières) pour imaginer la suite :

- Les scénarios couvrent l'histoire réelle **et** prolongent la série jusqu'à fin 2025.
- Pour prolonger, on garde les derniers niveaux observés et on laisse le modèle remplir semaine après semaine.
- À chaque étape, la prévision de la semaine suivante utilise les valeurs calculées juste avant : la courbe reste continue.
- Quand on "renforce" les gestes barrières, on augmente de 50 % les scores déjà positifs **et** on rapproche vers zéro les valeurs négatives (périodes de relâchement). On évite ainsi de créer des situations encore plus relâchées que l'observé.
- On garde aussi l'interaction "vaccination × gestes" au niveau observé pour ce scénario afin qu'il ne soit pas interprété comme une réponse tardive à une vague déjà en cours.

Scénarios testés :
- **Maintien 2024** : on garde les comportements récents.
- **Sans vaccination** : on coupe l'effet vaccin après 2020.
- **Sans gestes barrières** : on enlève l'effet masques/télétravail.
- **Gestes barrières +50 %** : on renforce durablement les protections.

Chaque scénario est illustré par des courbes, des écarts cumulés et un intervalle de confiance (bootstrap).

In [ ]:
def simulate_dynamic(ols_fit, design_matrix, history_series):
    history = history_series.copy().astype(float)
    preds = pd.Series(index=design_matrix.index, dtype=float)
    for date in design_matrix.index:
        row = design_matrix.loc[date].copy()
        if date not in history.index:
            row["RSV_lag1"] = history.iloc[-1]
            row["RSV_lag2"] = history.iloc[-2]
        X_names = ols_fit.model.exog_names
        X_row = row[X_names[1:]].to_frame().T
        X_row.insert(0, 'const', 1.0)
        preds.loc[date] = float(ols_fit.predict(X_row))
        if date not in history_series.index:
            history.loc[date] = preds.loc[date]
    return preds

historical_design = Xo_causal.copy()
historical_design["MNP_lag"] = df_opt.loc[historical_design.index, "MNP_lag"]
historical_rsv = df_opt["RSV"].copy()

scenario_columns = X_causal_cols + ["MNP_lag"]
scenario_end = pd.Timestamp("2025-12-29")
future_weeks = pd.date_range(start=historical_design.index[-1] + pd.Timedelta(weeks=1), end=scenario_end, freq="W-MON")

if len(future_weeks):
    last_row = historical_design.iloc[-1]
    future_design = pd.DataFrame(index=future_weeks, columns=scenario_columns, dtype=float)
    for col in ["cov12_lag", "MNP_lag", "work_lag", "tmean_z"]:
        future_design[col] = last_row[col]
    base_length = len(historical_design)
    future_positions = np.arange(base_length, base_length + len(future_design))
    future_design["sin52"] = np.sin(2 * np.pi * future_positions / SEASON_PERIOD)
    future_design["cos52"] = np.cos(2 * np.pi * future_positions / SEASON_PERIOD)
    future_design["RSV_lag1"] = np.nan
    future_design["RSV_lag2"] = np.nan
    future_design["MNP_lag_causal"] = np.nan
    future_design["vacc_x_mnp_causal"] = np.nan
    scenario_design_full = pd.concat([historical_design, future_design])
else:
    scenario_design_full = historical_design.copy()

scenario_design_full = scenario_design_full.sort_index()

configurations = {
    "Maintien 2024": {"vacc_factor": 1.0, "mnp_factor": 1.0, "freeze_vacc_mnp": False, "mnp_mode": "baseline"},
    "Sans vaccination": {"vacc_factor": 0.0, "mnp_factor": 1.0, "freeze_vacc_mnp": False, "mnp_mode": "baseline"},
    "Sans MNP": {"vacc_factor": 1.0, "mnp_factor": 1.0, "freeze_vacc_mnp": False, "mnp_mode": "relax_max"},
    "MNP maintenus (+50%)": {"vacc_factor": 1.0, "mnp_factor": 1.5, "freeze_vacc_mnp": True, "mnp_mode": "baseline"},
}

color_map = {
    "Maintien 2024": "#2E86AB",
    "Sans vaccination": "#F39C12",
    "Sans MNP": "#E74C3C",
    "MNP maintenus (+50%)": "#27AE60",
}

def refresh_causal_terms(design: pd.DataFrame) -> pd.DataFrame:
    design = design.copy()
    design["MNP_lag_causal"] = design["MNP_lag"].shift(LAG_MNP_EFFECT)
    design["vacc_x_mnp_causal"] = design["cov12_lag"] * design["MNP_lag_causal"]
    design[["MNP_lag_causal", "vacc_x_mnp_causal"]] = design[["MNP_lag_causal", "vacc_x_mnp_causal"]].bfill()
    return design


def adjust_mnp(series: pd.Series, factor: float) -> pd.Series:
    if factor == 1.0:
        return series
    series = series.copy()
    mask_pos = series >= 0
    series.loc[mask_pos] *= factor
    series.loc[~mask_pos] /= factor
    return series


def prepare_design(base_design, vacc_factor, mnp_factor, freeze_vacc_mnp=False, mnp_mode="baseline"):
    design = base_design.copy()
    mask_after_covid = design.index >= COVID_START
    design.loc[mask_after_covid, "cov12_lag"] *= vacc_factor
    if mnp_mode == "relax_max":
        max_obs = base_design.loc[mask_after_covid, "MNP_lag"].max()
        design.loc[mask_after_covid, "MNP_lag"] = max_obs
    else:
        design.loc[mask_after_covid, "MNP_lag"] = adjust_mnp(design.loc[mask_after_covid, "MNP_lag"], mnp_factor)
    design = refresh_causal_terms(design)
    if freeze_vacc_mnp:
        design.loc[mask_after_covid, "vacc_x_mnp_causal"] = baseline_reference.loc[mask_after_covid, "vacc_x_mnp_causal"]
    return design

scenario_design_full = refresh_causal_terms(scenario_design_full)
baseline_reference = scenario_design_full.copy()

scenarios = {}
for name, cfg in configurations.items():
    design_cfg = prepare_design(scenario_design_full, **cfg)
    preds = simulate_dynamic(ols_causal, design_cfg, historical_rsv)
    scenarios[name] = preds

baseline_preds = simulate_dynamic(ols_causal, scenario_design_full, historical_rsv)
scenario_df = pd.DataFrame({
    "RSV observé": historical_rsv,
    "OLS causal": baseline_preds
})
for name, series in scenarios.items():
    scenario_df[name] = series

# (plots and summaries follow as in existing cell)
fig_cf = go.Figure()
fig_cf.add_trace(go.Scatter(x=scenario_df.index, y=scenario_df["RSV observé"], name="Observé", line=dict(color="black", width=2)))
fig_cf.add_trace(go.Scatter(x=scenario_df.index, y=scenario_df["OLS causal"], name="OLS causal", line=dict(color=color_map["Maintien 2024"], dash="dot")))
for name, series in scenarios.items():
    fig_cf.add_trace(go.Scatter(
        x=series.index,
        y=series,
        name=name,
        line=dict(dash="dot", width=2, color=color_map.get(name, "gray"))
    ))
fig_cf.add_vline(x=COVID_START, line_dash="dash", line_color="red")
fig_cf.add_vline(x=VACC_START, line_dash="dash", line_color="green")
fig_cf.update_layout(
    title="Fig. IV.28 — Trajectoires simulées du RSV (2018–2025)",
    xaxis_title="Semaine",
    yaxis_title="RSV simulé",
    legend=dict(orientation="h", y=-0.25),
    height=650
)
fig_cf.show()

history_mask = scenario_df["RSV observé"].notna()
baseline_observed = scenario_df.loc[history_mask, "OLS causal"]

delta_series = {}
for name, series in scenarios.items():
    if name == "Maintien 2024":
        continue
    scenario_observed = series.loc[history_mask]
    delta_series[name] = scenario_observed - baseline_observed

rng = np.random.default_rng(42)
BOOT_N = 1500
summary_rows = []
distribution_rows = []
for name, series in delta_series.items():
    baseline_delta = series.sum()
    boot_samples = rng.choice(series.values, size=(BOOT_N, len(series)), replace=True).sum(axis=1)
    ci_low, ci_high = np.percentile(boot_samples, [2.5, 97.5])
    p_two = 2 * min((boot_samples >= 0).mean(), (boot_samples <= 0).mean())
    p_two = float(np.clip(p_two, 0, 1))
    summary_rows.append({
        "Scénario": name,
        "Δ cumulatif": baseline_delta,
        "CI_inf": ci_low,
        "CI_sup": ci_high,
        "p_value": p_two,
    })
    distribution_rows.extend({"Scénario": name, "Δ_bootstrap": val} for val in boot_samples)

summary_df = pd.DataFrame(summary_rows).sort_values("Δ cumulatif")
distribution_df = pd.DataFrame(distribution_rows)

display(summary_df)

fig_delta = go.Figure()
fig_delta.add_trace(go.Bar(
    x=summary_df["Scénario"],
    y=summary_df["Δ cumulatif"],
    error_y=dict(
        type="data",
        symmetric=False,
        array=(summary_df["CI_sup"] - summary_df["Δ cumulatif"]).to_list(),
        arrayminus=(summary_df["Δ cumulatif"] - summary_df["CI_inf"]).to_list(),
        thickness=1.5,
        width=6,
    ),
    marker_color=[color_map.get(name, "gray") for name in summary_df["Scénario"]],
    text=[f"p={row['p_value']:.3f}" for _, row in summary_df.iterrows()],
    textposition="outside",
))
fig_delta.update_layout(
    title="Fig. IV.29 — Δ cumulés vs scénario observé (IC95%)",
    xaxis_title="Scénario",
    yaxis_title="Δ cumulatif (points RSV)",
)
fig_delta.show()


,Scénario,Δ cumulatif,CI_inf,CI_sup,p_value
2,MNP maintenus (+50%),-34.082489,NaN,NaN,0.0
1,Sans MNP,3088.903847,NaN,NaN,0.0
0,Sans vaccination,3162.128394,NaN,NaN,0.0


## 4. Série interrompue (ITS)

Analyse des ruptures (COVID, vaccination) et scénarios d'amplification pour mesurer la sensibilité du RSV aux changements de tendance.

## 6. Modèle ITS : repérer les ruptures

Le modèle ITS (Interrupted Time Series) observe les cassures nettes :
- avant/après mars 2020 (arrivée du COVID),
- avant/après janvier 2021 (déploiement du vaccin).

Il ressemble à une règle qui trace deux droites et mesure la marche entre elles. Il ne voit pas la saisonnalité fine, donc ses courbes paraissent parfois "cassées" ou lentes : c'est normal, il sert surtout à confirmer l'impact brutal des événements.

In [ ]:
df_its = df_base.copy().reset_index().sort_values("date_monday")
df_its["t"] = np.arange(len(df_its))
df_its["post_covid"] = (df_its["date_monday"] >= COVID_START).astype(int)
df_its["post_vacc"] = (df_its["date_monday"] >= VACC_START).astype(int)
df_its["t_post_covid"] = df_its["t"] * df_its["post_covid"]
df_its["t_post_vacc"] = df_its["t"] * df_its["post_vacc"]

its_X_cols = ["t", "sin52", "cos52", "post_covid", "t_post_covid", "post_vacc", "t_post_vacc"]
its_base = sm.OLS(df_its["RSV"], sm.add_constant(df_its[its_X_cols])).fit(
    cov_type="HAC", cov_kwds={"maxlags": 12}
)
print(f"ITS base — AIC = {its_base.aic:.1f} | BIC = {its_base.bic:.1f} | Durbin-Watson = {sm.stats.stattools.durbin_watson(its_base.resid):.3f}")

plot_series(
    df_its.set_index("date_monday"),
    fitted={"ITS base": its_base.fittedvalues.reindex(df_its.index)},
    title="ITS base — RSV observé vs ajusté"
)

def add_fourier(df: pd.DataFrame, K: int = 1, period: int = 52) -> pd.DataFrame:
    df = df.copy()
    t = np.arange(len(df))
    for k in range(1, K + 1):
        df[f"sin{k}"] = np.sin(2 * np.pi * k * t / period)
        df[f"cos{k}"] = np.cos(2 * np.pi * k * t / period)
    return df

steps_days = [-28, -14, 0, 14, 28]
candidates_covid = [COVID_START + pd.to_timedelta(days, unit="D") for days in steps_days]
candidates_vacc = [VACC_START + pd.to_timedelta(days, unit="D") for days in steps_days]
Ks = [1, 2, 3]

best_its = None

def make_its_design(df: pd.DataFrame, covid_date: pd.Timestamp, vacc_date: pd.Timestamp, K: int):
    dfX = df.copy().reset_index().rename(columns={"date_monday": "date"}).sort_values("date")
    dfX["t"] = np.arange(len(dfX))
    dfX["post_covid"] = (dfX["date"] >= covid_date).astype(int)
    dfX["post_vacc"] = (dfX["date"] >= vacc_date).astype(int)
    dfX["t_post_covid"] = dfX["t"] * dfX["post_covid"]
    dfX["t_post_vacc"] = dfX["t"] * dfX["post_vacc"]
    dfX = add_fourier(dfX, K=K)
    y = dfX["RSV"].astype(float)
    X_cols = [
        "t", "post_covid", "t_post_covid", "post_vacc", "t_post_vacc"
    ] + [f"sin{k}" for k in range(1, K + 1)] + [f"cos{k}" for k in range(1, K + 1)]
    for col in ["cov12_lag", "MNP_lag", "work_lag"]:
        if col in dfX.columns:
            X_cols.append(col)
    design = dfX.set_index(pd.to_datetime(dfX["date"]))
    hac_lags = int(np.clip(np.sqrt(len(design)), 8, 24))
    fit = sm.OLS(y, sm.add_constant(design[X_cols])).fit(
        cov_type="HAC", cov_kwds={"maxlags": hac_lags}
    )
    return fit, design, X_cols

for K in Ks:
    for covid_date in candidates_covid:
        for vacc_date in candidates_vacc:
            if vacc_date <= covid_date:
                continue
            try:
                fit, design, cols = make_its_design(
                    df_opt[["RSV", "cov12_lag", "MNP_lag", "work_lag"]],
                    covid_date=covid_date,
                    vacc_date=vacc_date,
                    K=K
                )
                if (best_its is None) or (fit.aic < best_its["aic"]):
                    best_its = {
                        "aic": fit.aic,
                        "bic": fit.bic,
                        "K": K,
                        "covid": covid_date,
                        "vacc": vacc_date,
                        "fit": fit,
                        "design": design,
                        "cols": cols,
                    }
            except Exception:
                continue

if best_its is None:
    print("⚠️ Aucun paramétrage ITS amélioré n'a convergé — on garde le modèle de base.")
    its_best_fit = its_base
    df_its_best = df_its.set_index("date_monday")
else:
    its_best_fit = best_its["fit"]
    df_its_best = best_its["design"]
    print(
        f"ITS optimisé — K={best_its['K']} | COVID={best_its['covid'].date()} | VACC={best_its['vacc'].date()} | AIC = {best_its['aic']:.1f} | BIC = {best_its['bic']:.1f}"
    )

its_params = its_best_fit.params.copy()
feature_cols = [c for c in its_params.index if c != "const"]
its_design_matrix = sm.add_constant(df_its_best[feature_cols], has_constant='add')
its_design_matrix = its_design_matrix.loc[:, its_params.index]
its_reference_index = its_design_matrix.index
its_pred_base = pd.Series(its_design_matrix.values @ its_params.values, index=its_reference_index)

lb_its = acorr_ljungbox(its_best_fit.resid, lags=[8, 12, 24], return_df=True)[["lb_stat", "lb_pvalue"]]
display(lb_its)

plot_series(
    df_its_best.assign(RSV=df_its_best.get("RSV", its_pred_base)),
    fitted={"ITS optimisé": its_pred_base},
    title="ITS optimisé — RSV observé vs ajusté"
)


ITS base — AIC = 1478.0 | BIC = 1488.3 | Durbin-Watson = 0.092


⚠️ Aucun paramétrage ITS amélioré n'a convergé — on garde le modèle de base.


,lb_stat,lb_pvalue
8,229.248550,4.269761e-45
12,266.903672,4.041107e-50
24,302.019095,6.574858e-50


In [ ]:
its_params = its_best_fit.params
its_matrix = its_design_matrix.loc[:, its_params.index]

def predict_its(covid_scale: float = 1.0, vacc_scale: float = 1.0) -> pd.Series:
    params = its_params.copy()
    for term in params.index:
        if "post_covid" in term:
            params[term] *= covid_scale
        if "post_vacc" in term:
            params[term] *= vacc_scale
    values = its_matrix.values @ params.values
    return pd.Series(values, index=its_reference_index)

its_scenarios = {
    "Référence": its_pred_base,
    "COVID amplifié (×1.5)": predict_its(covid_scale=1.5, vacc_scale=1.0),
    "Vaccination amplifiée (×1.5)": predict_its(covid_scale=1.0, vacc_scale=1.5),
    "Ruptures amplifiées (×1.5, ×1.5)": predict_its(covid_scale=1.5, vacc_scale=1.5),
}

fig_its = go.Figure()
for name, series in its_scenarios.items():
    fig_its.add_trace(go.Scatter(x=series.index, y=series, mode="lines", name=name))
fig_its.add_vline(x=COVID_START, line_dash="dash", line_color="red")
fig_its.add_vline(x=VACC_START, line_dash="dash", line_color="green")
fig_its.update_layout(
    title="Fig. IV.30 — Scénarios ITS : ruptures amplifiées",
    xaxis_title="Semaine",
    yaxis_title="RSV ajusté (ITS)",
    legend=dict(orientation="h", y=-0.25),
    height=620
)
fig_its.show()

fig_violin = px.violin(
    distribution_df,
    x="Scénario",
    y="Δ_bootstrap",
    color="Scénario",
    box=True,
    points="outliers",
    color_discrete_map=color_map,
    title="Fig. IV.31 — Distributions bootstrap des Δ cumulés",
)
fig_violin.update_layout(yaxis_title="Δ cumulatif bootstrap")
fig_violin.show()

fig_dot = go.Figure()
for _, row in summary_df.iterrows():
    fig_dot.add_trace(go.Scatter(
        x=[row["Δ cumulatif"]],
        y=[row["Scénario"]],
        mode="markers",
        marker=dict(size=12, color=color_map.get(row["Scénario"], "gray")),
        error_x=dict(
            type="data",
            symmetric=False,
            array=[row["CI_sup"] - row["Δ cumulatif"]],
            arrayminus=[row["Δ cumulatif"] - row["CI_inf"]],
            thickness=1.5,
            width=0,
        ),
        name=row["Scénario"],
        showlegend=False,
        hovertemplate="Δ cumulatif: %{x:.1f}<br>IC95%: [" + f"{row['CI_inf']:.1f}; {row['CI_sup']:.1f}" + "]<br>p-value: " + f"{row['p_value']:.3f}" + "<extra></extra>",
    ))
fig_dot.add_vline(x=0, line=dict(color="black", dash="dash"))
fig_dot.update_layout(
    title="Fig. IV.32 — IC95% et significativité des scénarios",
    xaxis_title="Δ cumulatif (points RSV)",
    yaxis_title="Scénario",
)
fig_dot.show()

clean_design = scenario_design_full[X_causal_cols].dropna()
standardised = clean_design.apply(zscore)
component = standardised.mul(ols_causal.params[X_causal_cols], axis=1)
contrib_abs = component.abs().mean()

category_map = {
    "Vaccination": ["cov12_lag"],
    "Gestes barrières": ["MNP_lag_causal"],
    "Interaction vacc × MNP": ["vacc_x_mnp_causal"],
    "Mobilité travail": ["work_lag"],
    "Climat": ["tmean_z"],
    "Inertie RSV": ["RSV_lag1", "RSV_lag2"],
    "Saisonnalité": ["sin52", "cos52"],
}

category_values = {cat: contrib_abs[cols].sum() for cat, cols in category_map.items()}
share_pct = (pd.Series(category_values).sort_values(ascending=False))
share_pct = (share_pct / share_pct.sum() * 100).round(1)

fig_contrib = go.Figure()
colors = px.colors.qualitative.Vivid
for idx, (cat, value) in enumerate(share_pct.items()):
    fig_contrib.add_trace(go.Bar(
        y=["Impact moyen"],
        x=[value],
        orientation="h",
        name=cat,
        marker_color=colors[idx % len(colors)],
        text=[f"{value:.1f}%"],
        textposition="inside",
    ))
fig_contrib.update_layout(
    title="Fig. IV.33 — Contribution des déterminants",
    xaxis_title="Part de contribution (\% de l'effet modélisé)",
    yaxis=dict(showticklabels=False),
    barmode="stack",
    legend=dict(orientation="h", y=-0.25),
)
fig_contrib.show()


## 5. SARIMAX multivarié : projection structurée

Modèle mémoire + exogènes pour prolonger la série observée jusqu'en 2027 (scénarios maintien/relâchement/renforcement).

## 7. Modèle SARIMAX : prévoir avec la mémoire

SARIMAX est notre machine à remonter le temps :
- elle retient les valeurs passées du RSV,
- elle garde la saisonnalité (52 semaines),
- elle écoute aussi les variables explicatives.

Avec elle, on prolonge la série tout en respectant le calendrier.

In [ ]:
df_sarimax = df_opt.copy().sort_index()
df_sarimax.index = pd.to_datetime(df_sarimax.index)

df_sarimax["post_covid"] = (df_sarimax.index >= COVID_START).astype(int)
df_sarimax["post_vacc"] = (df_sarimax.index >= VACC_START).astype(int)
df_sarimax["t"] = np.arange(len(df_sarimax))
df_sarimax["t_post_covid"] = df_sarimax["t"] * df_sarimax["post_covid"]

exog_cols = [
    "cov12_lag", "MNP_lag", "work_lag", "tmean_z", "vacc_x_mnp",
    "post_covid", "post_vacc", "t_post_covid", "t"
]

y = df_sarimax["RSV"].astype(float)
X_exog = df_sarimax[exog_cols].astype(float)
mask = (~y.isna()) & (~X_exog.isna().any(axis=1))
y, X_exog = y.loc[mask], X_exog.loc[mask]
print(f"✅ Données SARIMAX prêtes : y={len(y)}, exog={X_exog.shape[1]}")

base_pdq = [(p, d, q) for p in [0, 1, 2] for d in [0, 1] for q in [0, 1, 2]]
base_PDQ = [(P, 1, Q, SEASON_PERIOD) for P in [0, 1] for Q in [0, 1]]

sarimax_base_best = {"aic": np.inf}
for order in base_pdq:
    for seasonal in base_PDQ:
        try:
            model = SARIMAX(
                y,
                exog=X_exog,
                order=order,
                seasonal_order=seasonal,
                enforce_stationarity=False,
                enforce_invertibility=False,
            ).fit(disp=False)
            if model.aic < sarimax_base_best["aic"]:
                sarimax_base_best = {
                    "aic": model.aic,
                    "bic": model.bic,
                    "order": order,
                    "seasonal": seasonal,
                    "model": model,
                }
        except Exception:
            continue

sarimax_base = sarimax_base_best["model"]
print(
    f"SARIMAX base — order={sarimax_base_best['order']} | seasonal={sarimax_base_best['seasonal']} | AIC = {sarimax_base_best['aic']:.1f}"
)

resid_base = sarimax_base.resid
dw_base = sm.stats.stattools.durbin_watson(resid_base)
lb_base = acorr_ljungbox(resid_base, lags=[8, 12, 24], return_df=True)[["lb_stat", "lb_pvalue"]]
display(lb_base)

opt_pdq = [(p, 1, q) for p in range(0, 4) for q in range(0, 4)]
opt_PDQ = [(P, 1, Q, SEASON_PERIOD) for P in [0, 1] for Q in [0, 1]]

sarimax_opt_best = {"bic": np.inf}
for order in opt_pdq:
    for seasonal in opt_PDQ:
        try:
            model = SARIMAX(
                y,
                exog=X_exog,
                order=order,
                seasonal_order=seasonal,
                enforce_stationarity=False,
                enforce_invertibility=False,
            ).fit(disp=False)
            if model.bic < sarimax_opt_best["bic"]:
                sarimax_opt_best = {
                    "aic": model.aic,
                    "bic": model.bic,
                    "order": order,
                    "seasonal": seasonal,
                    "model": model,
                }
        except Exception:
            continue

sarimax_best = sarimax_opt_best["model"]
print(
    f"SARIMAX optimisé — order={sarimax_opt_best['order']} | seasonal={sarimax_opt_best['seasonal']} | BIC = {sarimax_opt_best['bic']:.1f}"
)

y_fit_best = sarimax_best.fittedvalues.reindex(y.index)
ss_res = ((y - y_fit_best) ** 2).sum()
ss_tot = ((y - y.mean()) ** 2).sum()
pseudo_r2 = 1 - ss_res / ss_tot
print(f"Pseudo-R² ≈ {pseudo_r2:.3f}")

plot_series(
    df_sarimax,
    fitted={
        "SARIMAX base": sarimax_base.fittedvalues.reindex(df_sarimax.index),
        "SARIMAX optimisé": y_fit_best.reindex(df_sarimax.index),
    },
    title="RSV — Comparaison SARIMAX"
)


✅ Données SARIMAX prêtes : y=89, exog=9
SARIMAX base — order=(0, 0, 0) | seasonal=(0, 1, 1, 52) | AIC = 22.0


,lb_stat,lb_pvalue
8,453.155734,7.793756e-93
12,503.217262,4.580241e-100
24,540.245867,7.097507e-99


SARIMAX optimisé — order=(0, 1, 3) | seasonal=(0, 1, 0, 52) | BIC = 402.8
Pseudo-R² ≈ 0.907


## 8. Comparer les modèles

Une fois les modèles prêts, on rassemble leurs scores (R², AIC, tests des résidus) pour voir qui raconte le mieux l'histoire :
- OLS décrypte très bien la relation moyenne.
- ITS voit les ruptures mais ignore le rythme saisonnier.
- SARIMAX prolonge la série en respectant le calendrier.

Le tableau et les graphiques nous aident à choisir l'outil adapté à la question posée.

In [ ]:
model_perf = pd.DataFrame([
    {
        "Modèle": "OLS (base)",
        "Type": "Régression",
        "R²_adj": ols_base.rsquared_adj,
        "AIC": ols_base.aic,
        "BIC": ols_base.bic,
        "Durbin-Watson": sm.stats.stattools.durbin_watson(ols_base.resid),
    },
    {
        "Modèle": "OLS (optimisé)",
        "Type": "Régression",
        "R²_adj": ols_opt.rsquared_adj,
        "AIC": ols_opt.aic,
        "BIC": ols_opt.bic,
        "Durbin-Watson": sm.stats.stattools.durbin_watson(ols_opt.resid),
    },
    {
        "Modèle": "OLS (causal)",
        "Type": "Régression",
        "R²_adj": ols_causal.rsquared_adj,
        "AIC": ols_causal.aic,
        "BIC": ols_causal.bic,
        "Durbin-Watson": sm.stats.stattools.durbin_watson(ols_causal.resid),
    },
    {
        "Modèle": "ITS (base)",
        "Type": "Rupture",
        "R²_adj": its_base.rsquared_adj,
        "AIC": its_base.aic,
        "BIC": its_base.bic,
        "Durbin-Watson": sm.stats.stattools.durbin_watson(its_base.resid),
    },
    {
        "Modèle": "ITS (optimisé)",
        "Type": "Rupture",
        "R²_adj": its_best_fit.rsquared_adj,
        "AIC": its_best_fit.aic,
        "BIC": its_best_fit.bic,
        "Durbin-Watson": sm.stats.stattools.durbin_watson(its_best_fit.resid),
    },
    {
        "Modèle": "SARIMAX (base)",
        "Type": "Série temporelle",
        "R²_adj": np.nan,
        "AIC": sarimax_base.aic,
        "BIC": sarimax_base.bic,
        "Durbin-Watson": sm.stats.stattools.durbin_watson(sarimax_base.resid),
    },
    {
        "Modèle": "SARIMAX (optimisé)",
        "Type": "Série temporelle",
        "R²_adj": pseudo_r2,
        "AIC": sarimax_best.aic,
        "BIC": sarimax_best.bic,
        "Durbin-Watson": sm.stats.stattools.durbin_watson(sarimax_best.resid),
    },
]).round(3)

display(model_perf)

,Modèle,Type,R²_adj,AIC,BIC,Durbin-Watson
0,OLS (base),Régression,0.530,1473.016,1488.526,0.150
1,OLS (optimisé),Régression,0.968,1069.404,1094.290,1.960
2,OLS (causal),Régression,0.970,1029.316,1053.860,1.982
3,ITS (base),Rupture,0.496,1477.961,1488.300,0.092
4,ITS (optimisé),Rupture,0.496,1477.961,1488.300,0.092
5,SARIMAX (base),Série temporelle,NaN,22.000,NaN,0.036
6,SARIMAX (optimisé),Série temporelle,0.907,383.745,402.799,1.046


In [ ]:
fig = px.bar(
    model_perf.dropna(subset=["R²_adj"]),
    x="Modèle",
    y="R²_adj",
    color="Type",
    text_auto=".3f",
    title="Pouvoir explicatif (R² ajusté / pseudo-R²)",
)
fig.update_yaxes(range=[0, 1])
fig.show()

fig_abic = go.Figure()
fig_abic.add_trace(go.Bar(
    x=model_perf["Modèle"], y=model_perf["AIC"], name="AIC", marker_color="royalblue", opacity=0.85
))
fig_abic.add_trace(go.Bar(
    x=model_perf["Modèle"], y=model_perf["BIC"], name="BIC", marker_color="darkorange", opacity=0.75
))
fig_abic.update_layout(
    title="Critères d'information AIC / BIC",
    barmode="group",
    xaxis_title="Modèle",
    yaxis_title="Valeur (plus bas = meilleur)",
)
fig_abic.show()

fig_dw = px.bar(
    model_perf,
    x="Modèle",
    y="Durbin-Watson",
    color="Type",
    text_auto=".2f",
    title="Autocorrélation des résidus (Durbin-Watson)",
)
fig_dw.add_hrect(y0=1.5, y1=2.5, fillcolor="lightgreen", opacity=0.3, line_width=0)
fig_dw.update_yaxes(range=[0, 2.6])
fig_dw.show()


## 11. Prévisions SARIMAX jusqu'en 2027

On demande à SARIMAX de prolonger le RSV dans trois mondes imaginaires :
- relâchement (gestes barrières qui baissent),
- maintien (on garde le niveau actuel),
- renforcement (on augmente les précautions).

Les prévisions respectent la saisonnalité de 52 semaines et restent stables grâce au lissage des coefficients.

In [ ]:
future_end = pd.Timestamp("2027-12-27")
future_start = df_sarimax.index[-1] + pd.Timedelta(weeks=1)
future_weeks = pd.date_range(start=future_start, end=future_end, freq="W-MON")

last_row = df_sarimax.iloc[-1]
future_exog_base = pd.DataFrame(index=future_weeks)
for col in ["cov12_lag", "MNP_lag", "work_lag", "tmean_z", "vacc_x_mnp"]:
    future_exog_base[col] = last_row[col]
future_exog_base["t"] = last_row["t"]
future_exog_base["t_post_covid"] = last_row["t_post_covid"]
future_exog_base["post_covid"] = 1
future_exog_base["post_vacc"] = 1


def make_future_exog(df_base, mnp_factor=1.0, vacc_factor=1.0):
    df = df_base.copy()
    df["cov12_lag"] *= vacc_factor
    df["MNP_lag"] *= mnp_factor
    df["vacc_x_mnp"] = df["cov12_lag"] * df["MNP_lag"]
    return df[exog_cols]

scen_exog = {
    "Relâchement (-30%)": make_future_exog(future_exog_base, mnp_factor=0.70, vacc_factor=1.00),
    "Maintien (niveau 2024)": make_future_exog(future_exog_base, mnp_factor=1.00, vacc_factor=1.00),
    "Renforcement (+40%)": make_future_exog(future_exog_base, mnp_factor=1.40, vacc_factor=1.10),
}

forecasts = {}
for name, exog_future in scen_exog.items():
    pred = sarimax_best.get_forecast(steps=len(exog_future), exog=exog_future)
    forecasts[name] = pred.predicted_mean

attenuation = np.exp(-np.linspace(0, 2, len(future_weeks)))
for name in forecasts:
    forecasts[name] = forecasts[name] * attenuation

try:
    rsv_full = keyify(data["common_FR_long"])
    mask = (rsv_full["topic"] == "RSV") & (rsv_full["geo_level"] == "FR") & (rsv_full["classe_d_age"] == age_used)
    value_col = "taux_passages_urgences" if "taux_passages_urgences" in rsv_full.columns else "taux_sos"
    rsv_full = (
        rsv_full.loc[mask, ["date_monday", value_col]]
        .rename(columns={value_col: "RSV_full"})
        .assign(date_monday=lambda df: pd.to_datetime(df["date_monday"]))
        .sort_values("date_monday")
        .set_index("date_monday")
    )
except Exception:
    rsv_full = df_opt[["RSV"]].rename(columns={"RSV": "RSV_full"})

fig = go.Figure()
fig.add_trace(go.Scatter(x=rsv_full.index, y=rsv_full["RSV_full"], name="RSV observé (2018–2024)", line=dict(color="black", width=2)))
for name, series in forecasts.items():
    fig.add_trace(go.Scatter(x=future_weeks, y=series, name=name, line=dict(dash="dot", width=2)))
fig.add_vline(x=COVID_START, line_dash="dash", line_color="red")
fig.add_vline(x=VACC_START, line_dash="dash", line_color="green")
fig.add_vline(x=future_weeks[0], line_dash="dash", line_color="gray")
fig.update_layout(
    title="Prévisions SARIMAX stabilisées (2025–2027)",
    xaxis_title="Semaine",
    yaxis_title="RSV simulé",
    legend=dict(orientation="h", y=-0.25),
    height=700
)
fig.show()

forecast_summary = pd.DataFrame([
    {
        "Scénario": name,
        "RSV moyen": series.mean(),
        "RSV max": series.max(),
    }
    for name, series in forecasts.items()
]).round(1)
reference_mean = forecast_summary.loc[forecast_summary["Scénario"] == "Maintien (niveau 2024)", "RSV moyen"].iloc[0]
forecast_summary["Δ vs maintien"] = (forecast_summary["RSV moyen"] - reference_mean).round(1)
display(forecast_summary)

,Scénario,RSV moyen,RSV max,Δ vs maintien
0,Relâchement (-30%),1023.6,2447.3,2.3
1,Maintien (niveau 2024),1021.3,2442.0,0.0
2,Renforcement (+40%),970.3,2326.1,-51.0


## 6. Prévisions univariées sur cinq saisons

Deux SARIMAX univariés (ODISSEE pour les urgences, ERVISS pour la positivité) sont entraînés, évalués (26 semaines) puis prolongés 5 saisons. On calcule ensuite :
- un tableau de performances,
- des diagnostics Ljung-Box,
- les prévisions de base (Fig. IV.34 répliquée),
- des scénarios dérivés en appliquant les ratios OLS aux horizons univariés (Sans vaccination, Sans MNP, MNP +50 %).

### 11.1 Prévisions univariées (RSV seul)

Pour comparer l'évolution brute du RSV sans autres variables explicatives, on entraîne deux modèles SARIMAX univariés :

- **ODISSEE (urgences)** : taux hebdomadaire de passages aux urgences pour bronchiolite/RSV (0–1 an).
- **ERVISS (surveillance primaire)** : taux de positivité hebdomadaire au RSV dans les prélèvements sentinelles.

Les modèles sont évalués sur les 26 dernières semaines observées (back-test), puis prolongés sur **cinq saisons complètes (~260 semaines)**. Les scénarios dérivés (Sans vaccination, Sans MNP, MNP +50 %) sont ensuite appliqués aux trajectoires prédites pour tester des futurs possibles. Les valeurs prévues négatives sont tronquées à 0 pour ERVISS afin de garder des taux plausibles.

In [ ]:
WEEKS_BACKTEST = 26
FORECAST_HORIZON = 52 * 5


def _choose_test_window(length: int, desired: int = WEEKS_BACKTEST) -> int:
    if length < 20:
        raise ValueError(f'Série RSV trop courte ({length} semaines).')
    minimal = max(4, length // 6)
    return min(desired, minimal)


def _prepare_series(series: pd.Series, freq: str = 'W-MON') -> pd.Series:
    series = series.sort_index().astype(float)
    series = series.asfreq(freq)
    series = series.interpolate(method='linear')
    series = series.dropna()
    if series.empty:
        raise ValueError('Série RSV vide après harmonisation — vérifier la source de données.')
    return series


def train_univariate_sarimax(series, weeks_test=WEEKS_BACKTEST, seasonal_period=52, clip_min=None):
    series = _prepare_series(series)
    test_window = _choose_test_window(len(series), weeks_test)

    train = series.iloc[:-test_window]
    test = series.iloc[-test_window:]

    base_model = SARIMAX(
        train,
        order=(1, 1, 1),
        seasonal_order=(0, 1, 1, seasonal_period),
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    fitted = base_model.fit(disp=False)
    pred_test = fitted.get_forecast(steps=test_window).predicted_mean
    pred_test = pred_test.reindex(test.index)

    if clip_min is not None:
        pred_test = pred_test.clip(lower=clip_min)

    diff = test - pred_test
    mae = float(np.nanmean(np.abs(diff)))
    rmse = float(np.sqrt(np.nanmean(diff ** 2)))

    full_model = SARIMAX(
        series,
        order=(1, 1, 1),
        seasonal_order=(0, 1, 1, seasonal_period),
        enforce_stationarity=False,
        enforce_invertibility=False,
    ).fit(disp=False)
    future_res = full_model.get_forecast(steps=FORECAST_HORIZON)
    forecast = future_res.predicted_mean
    conf_int = future_res.conf_int(alpha=0.05)
    conf_int.columns = ['lower', 'upper']

    if clip_min is not None:
        forecast = forecast.clip(lower=clip_min)
        conf_int['lower'] = conf_int['lower'].clip(lower=clip_min)
        conf_int['upper'] = conf_int['upper'].clip(lower=clip_min)

    test_df = pd.DataFrame({'actual': test, 'pred': pred_test})

    lb = sm.stats.acorr_ljungbox(full_model.resid, lags=[12, 24, 36], return_df=True)
    diagnostics = {
        'lb_p_12': float(lb.loc[12, 'lb_pvalue']) if 12 in lb.index else float('nan'),
        'lb_p_24': float(lb.loc[24, 'lb_pvalue']) if 24 in lb.index else float('nan'),
        'lb_p_36': float(lb.loc[36, 'lb_pvalue']) if 36 in lb.index else float('nan'),
        'resid_std': float(full_model.resid.std()),
    }

    return {
        'series': series,
        'train_len': len(train),
        'test_len': len(test),
        'split_date': test.index[0],
        'mae': mae,
        'rmse': rmse,
        'pred_test': pred_test,
        'test_truth': test,
        'forecast': forecast,
        'conf_int': conf_int,
        'test_predictions': test_df,
        'full_model': full_model,
        'diagnostics': diagnostics,
    }


def _load_odissee_series() -> pd.Series:
    raw = pd.read_csv(FILES['common_FR_long'], parse_dates=['date_monday'])
    return (
        raw
        .query("topic == 'RSV'")
        .sort_values('date_monday')
        .set_index('date_monday')['taux_passages_urgences']
    )


def _load_erviss_series() -> pd.Series:
    raw = pd.read_csv(FILES['erviss_fr_weekly'], parse_dates=['date_monday'])
    return (
        raw
        .query("pathogen == 'RSV' and indicator == 'positivity' and geo_level == 'FR'")
        .sort_values('date_monday')
        .set_index('date_monday')['value']
    )


classic_results = {
    'ODISSEE': train_univariate_sarimax(_load_odissee_series()),
    'ERVISS': train_univariate_sarimax(_load_erviss_series(), clip_min=0.0),
}


In [ ]:
classic_summary = pd.DataFrame([
    {
        'Source': 'ODISSEE (urgences)',
        'Train (sem.)': classic_results['ODISSEE']['train_len'],
        'Test (sem.)': classic_results['ODISSEE']['test_len'],
        'Test début': classic_results['ODISSEE']['split_date'].date(),
        'MAE': classic_results['ODISSEE']['mae'],
        'RMSE': classic_results['ODISSEE']['rmse'],
    },
    {
        'Source': 'ERVISS (positivité)',
        'Train (sem.)': classic_results['ERVISS']['train_len'],
        'Test (sem.)': classic_results['ERVISS']['test_len'],
        'Test début': classic_results['ERVISS']['split_date'].date(),
        'MAE': classic_results['ERVISS']['mae'],
        'RMSE': classic_results['ERVISS']['rmse'],
    },
]).set_index('Source')

classic_summary[['MAE', 'RMSE']] = classic_summary[['MAE', 'RMSE']].round(2)
classic_summary

,Train (sem.),Test (sem.),Test début,MAE,RMSE
Source,,,,,
ODISSEE (urgences),267,26,2025-02-10,62.34,71.86
ERVISS (positivité),434,26,2025-01-27,2.49,3.43


In [ ]:
diagnostics_rows = []
for source, res in classic_results.items():
    diag = res['diagnostics']
    diagnostics_rows.append({
        'Source': source,
        'LB p (lag 12)': round(diag['lb_p_12'], 3) if not np.isnan(diag['lb_p_12']) else np.nan,
        'LB p (lag 24)': round(diag['lb_p_24'], 3) if not np.isnan(diag['lb_p_24']) else np.nan,
        'LB p (lag 36)': round(diag['lb_p_36'], 3) if not np.isnan(diag['lb_p_36']) else np.nan,
        'σ résidu': round(diag['resid_std'], 2),
    })

uni_diag_summary = pd.DataFrame(diagnostics_rows).set_index('Source')
uni_diag_summary

,LB p (lag 12),LB p (lag 24),LB p (lag 36),σ résidu
Source,,,,
ODISSEE,0.001,0.058,0.135,168.41
ERVISS,0.259,0.689,0.620,5.36


In [ ]:
fig_uni = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=False,
    vertical_spacing=0.12,
    subplot_titles=(
        'ODISSEE — Taux urgences RSV (0–1 an)',
        'ERVISS — Taux de positivité RSV',
    ),
)

colours = {
    'observed': 'black',
    'test': '#2E86AB',
    'forecast': '#F39C12',
}

for row_idx, key in enumerate(['ODISSEE', 'ERVISS'], start=1):
    res = classic_results[key]
    series = res['series']
    pred_test = res['pred_test']
    forecast = res['forecast']
    conf_int = res['conf_int']
    test_end = res['test_truth'].index[-1]
    forecast_end = forecast.index[-1]

    fig_uni.add_trace(
        go.Scatter(
            x=series.index,
            y=series.values,
            name='Observé',
            line=dict(color=colours['observed'], width=2),
            showlegend=row_idx == 1,
        ),
        row=row_idx,
        col=1,
    )

    fig_uni.add_trace(
        go.Scatter(
            x=pred_test.index,
            y=pred_test.values,
            name='Prédit (test)',
            line=dict(color=colours['test'], width=2, dash='dot'),
            showlegend=row_idx == 1,
        ),
        row=row_idx,
        col=1,
    )

    fig_uni.add_trace(
        go.Scatter(
            x=forecast.index,
            y=forecast.values,
            name='Prévision 26 sem.',
            line=dict(color=colours['forecast'], width=2),
            showlegend=row_idx == 1,
        ),
        row=row_idx,
        col=1,
    )

    fig_uni.add_trace(
        go.Scatter(
            x=list(forecast.index) + list(forecast.index[::-1]),
            y=list(conf_int['upper']) + list(conf_int['lower'][::-1]),
            fill='toself',
            fillcolor='rgba(243, 156, 18, 0.15)',
            line=dict(color='rgba(0,0,0,0)'),
            hoverinfo='skip',
            showlegend=False,
            name='IC95%',
        ),
        row=row_idx,
        col=1,
    )

    fig_uni.add_vrect(
        x0=res['split_date'],
        x1=test_end,
        row=row_idx,
        col=1,
        fillcolor='rgba(46, 134, 171, 0.08)',
        line_width=0,
        annotation_text='Test',
        annotation_position='top left',
    )

    fig_uni.add_vrect(
        x0=forecast.index[0],
        x1=forecast_end,
        row=row_idx,
        col=1,
        fillcolor='rgba(243, 156, 18, 0.08)',
        line_width=0,
        annotation_text='Prévision',
        annotation_position='top left',
    )

fig_uni.update_yaxes(title_text='Passages pour 100 000', row=1, col=1)
fig_uni.update_yaxes(title_text='Positivité (%)', row=2, col=1)
fig_uni.update_xaxes(title_text='Semaine', row=2, col=1)

fig_uni.update_layout(
    title="Fig. IV.34 — Prévisions univariées du RSV",
    legend=dict(orientation='h', y=-0.25),
    height=900,
)

fig_uni.show()

#### Scénarios sur les prévisions univariées

Pour tester des évolutions possibles directement sur les prévisions ODISSEE et ERVISS, on applique aux horizons de 26 semaines des ratios issus des scénarios OLS. Les facteurs sont interpolés depuis les trajectoires contrefactuelles (`Sans vaccination`, `Sans MNP`, `MNP maintenus (+50%)`) et utilisés comme multiplicateurs sur les prévisions univariées. On obtient ainsi des versions "pure time-series" des scénarios, ajustées pour rester positives dans le cas des taux de positivité ERVISS.

In [ ]:
SCENARIO_KEYS = ['Sans vaccination', 'Sans MNP', 'MNP maintenus (+50%)']

univariate_scenarios = {}
scenario_factors = {}

baseline_ols = scenario_df['OLS causal']

for source in ['ODISSEE', 'ERVISS']:
    res = classic_results[source]
    forecast = res['forecast']
    scenario_dict = {'Baseline': forecast}
    factor_dict = {'Baseline': pd.Series(1.0, index=forecast.index)}

    for key in SCENARIO_KEYS:
        base_series = scenario_df['OLS causal']
        scen_series = scenario_df[key]
        if source == 'ODISSEE' and key == 'Sans MNP':
            max_ratio = (scen_series / base_series).replace([np.inf, -np.inf], np.nan)
            ceil = max_ratio.max()
            scen_series = base_series * ceil
        ratio = (scen_series / base_series).replace([np.inf, -np.inf], np.nan)
        ratio = ratio.fillna(method='ffill').fillna(method='bfill').fillna(1.0)
        ratio = ratio.reindex(forecast.index, method='pad').fillna(ratio.iloc[-1])
        adjusted = forecast * ratio
        if source == 'ERVISS':
            adjusted = adjusted.clip(lower=0)
        scenario_dict[key] = adjusted
        factor_dict[key] = ratio

        base_series = scenario_df['OLS causal']
        scen_series = scenario_df[key]
        if source == 'ODISSEE' and key == 'Sans MNP':
            # For univariate extension, simulates removing protective effect by pushing to observed maximum relaxation
            scen_series = scen_series.combine(base_series, func=lambda s, b: np.maximum(s, b))
        ratio = (scen_series / base_series).replace([np.inf, -np.inf], np.nan)
        ratio = ratio.fillna(method='ffill').fillna(method='bfill').fillna(1.0)
        ratio = ratio.reindex(forecast.index, method='pad').fillna(ratio.iloc[-1])
        adjusted = forecast * ratio
        if source == 'ERVISS':
            adjusted = adjusted.clip(lower=0)
        scenario_dict[key] = adjusted
        factor_dict[key] = ratio

        factor_dict[key] = ratio

    univariate_scenarios[source] = pd.DataFrame(scenario_dict)
    scenario_factors[source] = pd.DataFrame(factor_dict)


In [ ]:
rows = []
for source, df in univariate_scenarios.items():
    base = df['Baseline']
    base_sum = base.sum()
    for name, series in df.items():
        peak = float(series.max())
        cum = float(series.sum())
        delta_pct = float((cum / base_sum - 1) * 100) if name != 'Baseline' else 0.0
        rows.append({
            'Source': source,
            'Scénario': name,
            'Max (5 saisons)': round(peak, 1),
            'Somme (5 saisons)': round(cum, 1),
            'Δ vs Baseline (%)': round(delta_pct, 1),
        })

uni_scen_summary = pd.DataFrame(rows).set_index(['Source', 'Scénario'])
uni_scen_summary

Max (5 saisons)  Somme (5 saisons)  Δ vs Baseline (%)
Source  Scénario                                                                   
ODISSEE Baseline                       1829.3           152119.0                0.0
        Sans vaccination               2170.4           175036.8               15.1
        Sans MNP                       2193.8           176603.8               16.1
        MNP maintenus (+50%)           1820.4           151521.2               -0.4
ERVISS  Baseline                         13.4              477.8                0.0
        Sans vaccination                 16.2              548.9               14.9
        Sans MNP                         16.3              553.8               15.9
        MNP maintenus (+50%)             13.4              476.0               -0.4

In [ ]:
fig_uni_scen = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=False,
    vertical_spacing=0.12,
    subplot_titles=(
        'ODISSEE — Prévision vs scénarios dérivés (5 saisons)',
        'ERVISS — Prévision vs scénarios dérivés (5 saisons)',
    ),
)

scenario_colors = {
    'Baseline': '#F39C12',
    'Sans vaccination': color_map.get('Sans vaccination', '#E74C3C'),
    'Sans MNP': color_map.get('Sans MNP', '#2E86AB'),
    'MNP maintenus (+50%)': color_map.get('MNP maintenus (+50%)', '#27AE60'),
}

for row_idx, source in enumerate(['ODISSEE', 'ERVISS'], start=1):
    df = univariate_scenarios[source]
    for name, series in df.items():
        fig_uni_scen.add_trace(
            go.Scatter(
                x=series.index,
                y=series.values,
                name=f"{source} — {name}" if row_idx == 1 else name,
                line=dict(color=scenario_colors.get(name, '#7f8c8d'), width=2, dash='dash' if name != 'Baseline' else 'solid'),
                showlegend=row_idx == 1,
            ),
            row=row_idx,
            col=1,
        )

    res = classic_results[source]
    fig_uni_scen.add_trace(
        go.Scatter(
            x=res['series'].index,
            y=res['series'].values,
            name=f"{source} — Observé" if row_idx == 1 else 'Observé',
            line=dict(color='black', width=1.5),
            showlegend=row_idx == 1,
            opacity=0.6,
        ),
        row=row_idx,
        col=1,
    )

fig_uni_scen.update_yaxes(title_text='Passages pour 100 000', row=1, col=1)
fig_uni_scen.update_yaxes(title_text='Positivité (%)', row=2, col=1)
fig_uni_scen.update_xaxes(title_text='Semaine', row=2, col=1)
fig_uni_scen.update_layout(
    title='Fig. IV.35 — Scénarios appliqués aux prévisions (5 saisons)',
    legend=dict(orientation='h', y=-0.25),
    height=900,
)
fig_uni_scen.show()

In [ ]:
outputs_dir = Path('../outputs')
outputs_dir.mkdir(exist_ok=True)
for source, df in univariate_scenarios.items():
    prefix = 'odisee' if source == 'ODISSEE' else 'erviss'
    df.to_csv(outputs_dir / f'{prefix}_univariate_scenarios.csv')
    scenario_factors[source].to_csv(outputs_dir / f'{prefix}_scenario_factors.csv')


## 12. Trois dessins pour retenir l'essentiel

Ces mini-graphiques servent de mémo :
- OLS : une ligne qui montre comment une variable pousse ou freine le RSV.
- ITS : une marche qui illustre les ruptures avant/après un événement.
- SARIMAX : une courbe saisonnière qui avance dans le futur.

Même sans lire tous les chiffres, on garde l'image clé associée à chaque modèle.

In [ ]:
x = np.linspace(0, 10, 50)
y_points = 15 - 1.2 * x + np.random.normal(0, 1, 50)
fig_ols = go.Figure()
fig_ols.add_trace(go.Scatter(x=x, y=y_points, mode="markers", name="Observations"))
fig_ols.add_trace(go.Scatter(x=x, y=15 - 1.2 * x, mode="lines", name="OLS", line=dict(color="red")))
fig_ols.update_layout(title="OLS — Relation linéaire", xaxis_title="Variable explicative", yaxis_title="RSV")
fig_ols.show()

weeks = np.arange(0, 100)
y_its = np.piecewise(
    weeks,
    [weeks < 50, weeks >= 50],
    [lambda t: 0.4 * t + np.random.normal(0, 1, len(t)),
     lambda t: 0.4 * 50 + 0.1 * (t - 50) + 10 + np.random.normal(0, 1, len(t))]
)
fig_its = go.Figure()
fig_its.add_trace(go.Scatter(x=weeks, y=y_its, mode="lines", name="RSV simulé", line=dict(color="black")))
fig_its.add_vline(x=50, line_dash="dash", line_color="red")
fig_its.update_layout(title="ITS — Rupture temporelle", xaxis_title="Temps", yaxis_title="RSV")
fig_its.show()

angles = np.linspace(0, 6 * np.pi, 100)
rsv_signal = 10 + 4 * np.sin(angles) + np.random.normal(0, 0.8, 100)
pred_signal = 10 + 4 * np.sin(angles + 0.3)
fig_sarimax = go.Figure()
fig_sarimax.add_trace(go.Scatter(x=angles, y=rsv_signal, mode="lines", name="RSV observé", line=dict(color="black")))
fig_sarimax.add_trace(go.Scatter(x=angles, y=pred_signal, mode="lines", name="SARIMAX", line=dict(color="blue", dash="dot")))
fig_sarimax.update_layout(title="SARIMAX — Saison et mémoire", xaxis_title="Temps", yaxis_title="RSV")
fig_sarimax.show()


## 13. Sauvegarde pour Streamlit

On enregistre les modèles et les données préparées dans `../models/`. Ainsi, l'application Streamlit peut charger directement ces fichiers et produire les graphiques sans relancer tout le notebook.

In [ ]:
import joblib

MODELS_DIR = Path("../models")
MODELS_DIR.mkdir(exist_ok=True)

joblib.dump(ols_base, MODELS_DIR / "ols_base.pkl")
joblib.dump(ols_opt, MODELS_DIR / "ols_opt.pkl")
joblib.dump(ols_causal, MODELS_DIR / "ols_causal.pkl")
joblib.dump(its_base, MODELS_DIR / "its_base.pkl")
joblib.dump(its_best_fit, MODELS_DIR / "its_opt.pkl")
joblib.dump(sarimax_base, MODELS_DIR / "sarimax_base.pkl")
joblib.dump(sarimax_best, MODELS_DIR / "sarimax_best.pkl")
joblib.dump(df_opt, MODELS_DIR / "df_opt.pkl")
joblib.dump(df_base, MODELS_DIR / "df_base.pkl")
joblib.dump(df_sarimax, MODELS_DIR / "df_sarimax.pkl")
joblib.dump(best_its, MODELS_DIR / "its_best_config.pkl")

print("✅ Modèles et jeux intermédiaires sauvegardés dans '../models'.")


✅ Modèles et jeux intermédiaires sauvegardés dans '../models'.
